<a href="https://colab.research.google.com/github/LeonGoergen/informationExtraction/blob/main/data_prep/DataPreparationMobility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from tqdm import tqdm
import json
import unicodedata
import os
from collections import Counter

# Data Preparation

In [ ]:
raw1 = pd.read_json('/content/drive/MyDrive/Masterarbeit/Mobility/raw1.jsonl', lines=True)
raw2 = pd.read_json('/content/drive/MyDrive/Masterarbeit/Mobility/raw2.jsonl', lines=True)
df = pd.concat([raw1, raw2])

df

,id,raw,text,uri,signature,langCode,date,tokens,sentences,corefChains,conceptMentions,relationMentions,annotations,provenance,refids,title,attributes,docType
0,755276242237218816,"{""in_reply_to_status_id_str"":null,""in_reply_to...",Werde meine #Verspätungen als #Statistik mal v...,https://twitter.com/2172146480/status/75527624...,NaN,de,2016-07-19T05:41:03Z,"[{'span': {'start': 0, 'end': 5, 'attributes':...","[{'id': '755276242237218816::0', 'span': {'sta...",NaN,[{'id': 'c/008855e1-df4e-4aac-9bfb-0027f43f8dd...,[],NaN,None,"{'fileName': 'smartdata_tweet_1311.xml', 'sour...",None,"{'crawlTimestamp': '2016-07-19T05:41:03Z', 'Re...",TWITTER_JSON
1,1108129826844672001,"{""extended_tweet"":{""entities"":{""urls"":[],""hash...",#S4 #RegioNDS #Teilausfall #Mellendorf(23.03)>...,https://twitter.com/814393643125604352/status/...,NaN,de,2019-03-19T23:15:13.000+01:00,"[{'span': {'start': 0, 'end': 3, 'attributes':...","[{'id': '1108129826844672001::0', 'span': {'st...",NaN,[{'id': 'c/c7e02c93-f2e7-4329-ba1b-83e37bfcfc5...,[{'id': 'r/3236118d-2b3e-4621-8e81-4371a31812b...,NaN,"[{'annotator': 'WebAnno3/tatjana', 'license': ...",{'inception.filename': 'dcc9286ccef1cc31'},,{},TWITTER_JSON
2,https://www.presseportal.de/blaulicht/pm/62288...,POL-WI: Falscher Polizeibeamter auf A66 festge...,POL-WI: Falscher Polizeibeamter auf A66 festge...,https://www.presseportal.de/blaulicht/pm/62288...,NaN,de,2018-11-15T22:07:09.000+01:00,"[{'span': {'start': 0, 'end': 3, 'attributes':...",[{'id': 'https://www.presseportal.de/blaulicht...,NaN,[{'id': 'c/177fd54f-6901-4d8f-9a85-e2a2ecedcdd...,[],NaN,"[{'annotator': 'WebAnno3/tatjana', 'license': ...",{'inception.filename': 'b2487eeaefd2d427'},POL-WI: Falscher Polizeibeamter auf A66 festge...,{},RSS_XML
3,http://www.ndr.de/nachrichten/verkehr/index.ht...,A28: Gefahr!.\n\nVorsicht bitte auf der A28 Le...,Vorsicht bitte auf der A28 Leer Richtung Olden...,http://www.ndr.de/nachrichten/verkehr/index.html,NaN,de,2016-05-04T19:02:14Z,"[{'span': {'start': 0, 'end': 8, 'attributes':...",[{'id': 'http://www.ndr.de/nachrichten/verkehr...,NaN,[{'id': 'c/8ae82956-cf9e-4d05-afc3-e351bb70103...,[],NaN,None,"{'fileName': 'smartdata_rss_879.xml', 'source'...",None,"{'crawlTimestamp': '2016-05-04T19:02:14Z', 'Re...",RSS_XML
4,1111089830971215872,"{""extended_tweet"":{""entities"":{""urls"":[],""hash...",1 / 2 #S7: weiterhin bestehende Weichenstörung...,https://twitter.com/166866601/status/111108983...,NaN,de,2019-03-28T03:17:13.000+01:00,"[{'span': {'start': 0, 'end': 1, 'attributes':...","[{'id': '1111089830971215872::0', 'span': {'st...",NaN,[{'id': 'c/636eeb03-35c4-4250-8765-ac36e9664e7...,[],NaN,"[{'annotator': 'WebAnno3/tatjana', 'license': ...",{'inception.filename': '3fdf168e889110cd'},,{},TWITTER_JSON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,666230710164811777,"{""in_reply_to_status_id_str"":null,""in_reply_to...",POL-IZ: 151116.3 Kellinghusen: Unfall in der L...,None,NaN,de,2015-11-16T12:25:53Z,"[{'span': {'start': 0, 'end': 3, 'attributes':...","[{'id': '666230710164811777::0', 'span': {'sta...",NaN,[{'id': 'c/b44c29ed-cd52-4b7a-a4a5-2bf08fc61bc...,[{'id': 'r/a88924e0-55cb-429f-9228-f8ea924c265...,NaN,NaN,"{'fileName': 'smartdata_tweet_656.xml', 'sourc...",NaN,"{'crawlTimestamp': '2015-11-16T12:25:53Z', 'Re...",TWITTER_JSON
619,http://bauarbeiten.bahn.de/fernverkehr/Linie/I...,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<Doc...","am Dienstag, 2. August, 3.00 – 3.45 Uhr\nMeldu...",http://bauarbeiten.bahn.de/fernverkehr/Linie/I...,NaN,de,2016-06-27T16:25:53.03Z,"[{'span': {'start': 0, 'end': 2, 'attributes':...",[{'id': 'http://bauarbeiten.bahn.de/fernverkeh...,NaN,[{'id': 'c/15d455f0-0660-4579-ab5c-4cbdd5d9254...,[{'id': 'r/eb26fa73-a1f3-4d6e-8dc0-11fd57ae2e7...,NaN,NaN,"{'fileName': 'smartdata_rss_857.xml', 'source'...",NaN,"{'crawlTimestamp': '2016-06-27T16:25:53.03Z', ...",RSS_XML
620,http://www.ndr.de/nachrichten/verkehr/index.ht...,A27: Stau.\n\nA27 Bremen Richtung Br

In [ ]:
print(df.iloc[1].text)
print(df.iloc[1].tokens)

#S4 #RegioNDS #Teilausfall #Mellendorf(23.03)> #Bennemühlen(23.07).  Grund: technische Störung an der Strecke. Bitte nutzen Sie #RB38 nach Soltau über Bennemühlen Abfahrt: 23:08 Uhr vom Gleis 2
[{'span': {'start': 0, 'end': 3, 'attributes': None}, 'lemma': '#s4', 'ner': 'B-location-route', 'posTag': 'CARD', 'attributes': {'IsNewlineAnnotation': 'false'}}, {'span': {'start': 4, 'end': 13, 'attributes': None}, 'lemma': '#regionds', 'ner': 'B-organization-company', 'posTag': 'NN', 'attributes': {'IsNewlineAnnotation': 'false'}}, {'span': {'start': 14, 'end': 26, 'attributes': None}, 'lemma': '#teilausfall', 'ner': 'B-trigger', 'posTag': 'NN', 'attributes': {'IsNewlineAnnotation': 'false'}}, {'span': {'start': 27, 'end': 38, 'attributes': None}, 'lemma': '#mellendorf', 'ner': 'B-location-stop', 'posTag': 'NE', 'attributes': {'IsNewlineAnnotation': 'false'}}, {'span': {'start': 38, 'end': 39, 'attributes': None}, 'lemma': '-lrb-', 'ner': 'O', 'posTag': 'TRUNC', 'attributes': {'IsNewlineAnno

In [ ]:
def extract_tokens_and_ner(row):
    tokens = [token['lemma'] for token in row]
    ner_tags = [token['ner'] for token in row]
    return pd.Series([tokens, ner_tags])

df[['tokens', 'ner_tags']] = df['tokens'].apply(extract_tokens_and_ner)
df = df[['tokens', 'ner_tags']]
df

,tokens,ner_tags
0,"[werde, meine, #verspätungen, als, #statistik,...","[O, O, O, O, O, O, O, O, B-organization-compan..."
1,"[#s4, #regionds, #teilausfall, #mellendorf, -l...","[B-location-route, B-organization-company, B-t..."
2,"[pol, -, wi, :, falscher, polizeibeamter, auf,...","[B-organization, I-organization, I-organizatio..."
3,"[vorsicht, bitte, auf, der, a28, leer, richtun...","[O, O, O, O, B-location-street, B-location-cit..."
4,"[1, /, 2, #s7, :, weiterhin, bestehende, weich...","[B-number, O, B-number, B-location-route, O, O..."
...,...,...
618,"[pol, -, iz, :, 151116.3, kellinghusen, :, unf...","[B-organization, I-organization, I-organizatio..."
619,"[am, dienstag, ,, 2, ., august, ,, 3.00, --, 3...","[O, B-date, I-date, I-date, I-date, I-date, O,..."
620,"[a27, bremen, richtung, bremerhaven, zwischen,...","[B-location-street, B-location-city, O, B-loca..."
621,"[rt, @rbbabendschau, :, schlechte, nachrichten...","[O, B-organization-company, O, O, O, O, O, O, ..."


In [ ]:
print(df.iloc[1].tokens)
print(df.iloc[1].ner_tags)

['#s4', '#regionds', '#teilausfall', '#mellendorf', '-lrb-', '23.03', '-rrb-', '>', '#bennemühlen', '-lrb-', '23.07', '-rrb-', '.', 'grund', ':', 'technische', 'störung', 'a', 'der', 'strecke', '.', 'bitte', 'nutzen', 'sie', '#rb38', 'nach', 'soltau', 'über', 'bennemühlen', 'abfahrt', ':', '23:08', 'uhr', 'vom', 'gleis', '2']
['B-location-route', 'B-organization-company', 'B-trigger', 'B-location-stop', 'O', 'B-date', 'O', 'O', 'B-location-city', 'O', 'B-date', 'O', 'O', 'O', 'O', 'B-event-cause', 'I-event-cause', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location-route', 'O', 'B-location-city', 'O', 'B-location-stop', 'O', 'O', 'B-time', 'I-time', 'O', 'O', 'B-number']


In [ ]:
def strip_prefix(tag):
    if tag.startswith('B-') or tag.startswith('I-'):
        return tag[2:]
    return tag

all_ner_tags = [strip_prefix(tag) for tags in df['ner_tags'] for tag in tags]

tag_counts = Counter(all_ner_tags)
tag_counts

Counter({'O': 51212,
         'organization-company': 2113,
         'location-route': 2778,
         'trigger': 2723,
         'location-stop': 5010,
         'date': 1840,
         'location-city': 1996,
         'event-cause': 1083,
         'time': 1389,
         'number': 694,
         'organization': 669,
         'location-street': 1362,
         'location': 3056,
         'duration': 1112,
         'person': 151,
         'set': 45,
         'money': 29,
         'distance': 402,
         'disaster-type': 92,
         'org-position': 4,
         'percent': 2})

In [ ]:
train, test = train_test_split(df, test_size=0.15, random_state=1)
len(train), len(test)

(2327, 411)

In [ ]:
train_json = train.to_json(orient='records')
test_json = test.to_json(orient='records')

with open('/content/drive/MyDrive/Masterarbeit/Mobility/train.json', 'w') as f:
  f.write(train_json)

with open('/content/drive/MyDrive/Masterarbeit/Mobility/test.json', 'w') as f:
  f.write(test_json)